In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
# Criação da sessão Spark
spark = SparkSession.builder \
    .appName("PySpark BigQuery Connection") \
    .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2') \
    .config("spark.jars", "/usr/local/lib/spark-connectors/bigquery-connector-hadoop2-latest.jar") \
    .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true -Dio.netty.noUnsafe=true") \
    .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true -Dio.netty.noUnsafe=true") \
    .getOrCreate()

24/09/28 16:03:14 WARN Utils: Your hostname, spark-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/09/28 16:03:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0d35e162-324f-4e3e-9439-8ab517d0bc24;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 in central
:: resolution report :: resolve 804ms :: artifacts dl 13ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	------------------------------------------------------------------

In [3]:
spark.conf.set("viewsEnabled", True)
spark.conf.set("materializationDataset", "SOT")

In [4]:
sc = spark.sparkContext
sc.setLogLevel("INFO")
sc._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
sc._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.json.keyfile', '/usr/local/lib/gcp/credentials/my-project-1508437523553-e9bafe7e3368.json')

In [5]:
# Função para salvar DataFrame em formato Parquet
def save_to_bigquery(df, dataset, table_name):
    # Salva o DataFrame em formato Parquet
    df.write \
    .format("bigquery") \
    .option("table", f"{dataset.upper()}.{table_name}") \
    .option("temporaryGcsBucket", "meu-bucket-temporario-spark") \
    .option("credentialsFile", "/usr/local/lib/gcp/credentials/my-project-1508437523553-e9bafe7e3368.json") \
    .mode("overwrite") \
    .save()

In [6]:
# Função para ler dados do BigQuery
def read_from_bigquery(dataset, table_name):  
    df = spark.read \
        .format('bigquery') \
        .option('table', f"{dataset.upper()}.{table_name}") \
        .option("credentialsFile", "/usr/local/lib/gcp/credentials/my-project-1508437523553-e9bafe7e3368.json") \
        .load()

    return df

In [7]:
df = read_from_bigquery('SOT', 'tbx001_data')
df.createOrReplaceTempView("tbx001_data")

In [8]:
df.show()

24/09/28 16:04:34 INFO DirectBigQueryRelation: |Querying table my-project-1508437523553.SOT.tbx001_data, parameters sent from Spark:|requiredColumns=[uf,ano,mes,semana,ano_nascimento,sexo,cor_raca,tipo_area,escolaridade,teve_febre,teve_dificuldade_respirar,teve_dor_cabeca,teve_fadiga,teve_perda_cheiro,foi_posto_saude,ficou_em_casa,ficou_internado,resultado_covid,tem_plano_saude,faixa_rendimento,situacao_domicilio],|filters=[]
24/09/28 16:04:39 INFO ReadSessionCreator: Read session:{"readSessionName":"projects/my-project-1508437523553/locations/us-east1/sessions/CAISDGxtR1UtNjlqMGxEYRoCdngaAnVo","readSessionCreationStartTime":"2024-09-28T19:04:34.272Z","readSessionCreationEndTime":"2024-09-28T19:04:38.979Z","readSessionPrepDuration":2423,"readSessionCreationDuration":2284,"readSessionDuration":4707}
24/09/28 16:04:39 INFO ReadSessionCreator: Requested 20000 max partitions, but only received 4 from the BigQuery Storage API for session projects/my-project-1508437523553/locations/us-east1/

+-----+----+---+------+--------------+-----+--------+---------+--------------------+----------+-------------------------+---------------+-----------+-----------------+---------------+-------------+---------------+---------------+---------------+----------------+------------------+
|   uf| ano|mes|semana|ano_nascimento| sexo|cor_raca|tipo_area|        escolaridade|teve_febre|teve_dificuldade_respirar|teve_dor_cabeca|teve_fadiga|teve_perda_cheiro|foi_posto_saude|ficou_em_casa|ficou_internado|resultado_covid|tem_plano_saude|faixa_rendimento|situacao_domicilio|
+-----+----+---+------+--------------+-----+--------+---------+--------------------+----------+-------------------------+---------------+-----------+-----------------+---------------+-------------+---------------+---------------+---------------+----------------+------------------+
|Amapá|2020| 05|     1|          9999|Homem|   Parda|  Capital|Fundamental incom...|       Sim|                     Não |           Não |       Não |     

24/09/28 16:04:52 INFO CodeGenerator: Code generated in 169.636727 ms
